# SIFT+SVM

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random
import pickle
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import *
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

In [2]:
X_train=pickle.load(open("X_train","rb"))[0:2000]
X_test=pickle.load(open("X_test","rb"))[0:400]
y_train=pickle.load(open("y_train","rb"))[0:2000]
y_test=pickle.load(open("y_test","rb"))[0:400]

In [3]:
X_train.shape

(2000, 128, 128, 1)

In [4]:
sift = cv.SIFT_create()

In [5]:
X_train_descriptors = []
y_train_labels = []
for i in range(X_train.shape[0]):
    image = X_train[i].astype(np.uint8)
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is None:
        continue
    descriptors /= np.sqrt(np.sum(descriptors**2, axis=1)).reshape(-1, 1)
    X_train_descriptors.append(descriptors)
    y_train_labels.append(np.repeat(y_train[i], len(keypoints)))
X_train_descriptors = np.concatenate(X_train_descriptors, axis=0)
y_train_labels = np.concatenate(y_train_labels, axis=0)

print(X_train_descriptors.shape)
print(y_train_labels.shape)

pickle_out=open("X_train_descriptors","wb")
pickle.dump(X_train_descriptors,pickle_out)
pickle_out.close()

pickle_out=open("y_train_labels","wb")
pickle.dump(y_train_labels,pickle_out)
pickle_out.close()


(297335, 128)
(297335,)


In [6]:
X_test_descriptors = []
y_test_labels = []
for i in range(X_test.shape[0]):
    image = X_test[i].astype(np.uint8)
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is None:
        continue
    descriptors /= np.sqrt(np.sum(descriptors**2, axis=1)).reshape(-1, 1)
    X_test_descriptors.append(descriptors)
    y_test_labels.append(np.repeat(y_test[i], len(keypoints)))
X_test_descriptors = np.concatenate(X_test_descriptors, axis=0)
y_test_labels = np.concatenate(y_test_labels, axis=0)

pickle_out=open("X_test_descriptors","wb")
pickle.dump(X_test_descriptors,pickle_out)
pickle_out.close()

pickle_out=open("y_test_labels","wb")
pickle.dump(y_test_labels,pickle_out)
pickle_out.close()

In [7]:
#choose part to train
shuffle_index = np.random.permutation(X_train_descriptors.shape[0])
X_train_descriptors=X_train_descriptors[shuffle_index]
y_train_labels=y_train_labels[shuffle_index]
X_train_descriptors=X_train_descriptors[0:10000]
y_train_labels=y_train_labels[0:10000]

In [8]:
X_train_descriptors.shape

(10000, 128)

In [10]:
def sv(ker,C):
    clf = svm.SVC(kernel=ker,C=C)
    para={}
    clf.fit(X_train_descriptors, y_train_labels)
    y_pred = clf.predict(X_test_descriptors)
    print("Accuracy:",accuracy_score(y_test_labels, y_pred))
    print("Precision:",precision_score(y_test_labels, y_pred,average='weighted'))
    print("Recall:",recall_score(y_test_labels, y_pred,average='weighted'))
    print("F1:",f1_score(y_test_labels, y_pred,average='weighted'))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test_labels, y_pred))
    print("Classification Report:")
    print(classification_report(y_test_labels, y_pred))
    para['acc']=accuracy_score(y_test_labels, y_pred)
    para['pre']=precision_score(y_test_labels, y_pred,average='weighted')
    para['rec']=recall_score(y_test_labels, y_pred,average='weighted')
    para['f1']=f1_score(y_test_labels, y_pred,average='weighted')
    para['cm']=confusion_matrix(y_test_labels, y_pred)
    para['cr']=classification_report(y_test_labels, y_pred)
    return para

In [11]:
sl001=sv('linear',0.01)

Accuracy: 0.5110897953688746
Precision: 0.26121277893019806
Recall: 0.5110897953688746
F1: 0.34572767247949415
Confusion Matrix:
[[30371     0]
 [29053     0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      1.00      0.68     30371
           1       0.00      0.00      0.00     29053

    accuracy                           0.51     59424
   macro avg       0.26      0.50      0.34     59424
weighted avg       0.26      0.51      0.35     59424



D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: 

In [12]:
sl01=sv('linear',0.1)

Accuracy: 0.5225329833064082
Precision: 0.5230043538981372
Recall: 0.5225329833064082
F1: 0.48739067490822935
Confusion Matrix:
[[23618  6753]
 [21620  7433]]
Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.78      0.62     30371
           1       0.52      0.26      0.34     29053

    accuracy                           0.52     59424
   macro avg       0.52      0.52      0.48     59424
weighted avg       0.52      0.52      0.49     59424



In [13]:
sl1=sv('linear',1)

Accuracy: 0.5214391491653204
Precision: 0.5200623294984749
Recall: 0.5214391491653204
F1: 0.5144028515189734
Confusion Matrix:
[[19409 10962]
 [17476 11577]]
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.64      0.58     30371
           1       0.51      0.40      0.45     29053

    accuracy                           0.52     59424
   macro avg       0.52      0.52      0.51     59424
weighted avg       0.52      0.52      0.51     59424



In [14]:
sl10=sv('linear',10)

Accuracy: 0.5180398492191707
Precision: 0.5170015839204603
Recall: 0.5180398492191707
F1: 0.5156768672653333
Confusion Matrix:
[[17821 12550]
 [16090 12963]]
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.59      0.55     30371
           1       0.51      0.45      0.48     29053

    accuracy                           0.52     59424
   macro avg       0.52      0.52      0.51     59424
weighted avg       0.52      0.52      0.52     59424



In [15]:
sl100=sv('linear',100)

Accuracy: 0.5174845180398492
Precision: 0.5165640086333088
Recall: 0.5174845180398492
F1: 0.5156878440577966
Confusion Matrix:
[[17543 12828]
 [15845 13208]]
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.58      0.55     30371
           1       0.51      0.45      0.48     29053

    accuracy                           0.52     59424
   macro avg       0.52      0.52      0.51     59424
weighted avg       0.52      0.52      0.52     59424



In [16]:
sr001=sv('rbf',0.01)

Accuracy: 0.5110897953688746
Precision: 0.26121277893019806
Recall: 0.5110897953688746
F1: 0.34572767247949415
Confusion Matrix:
[[30371     0]
 [29053     0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      1.00      0.68     30371
           1       0.00      0.00      0.00     29053

    accuracy                           0.51     59424
   macro avg       0.26      0.50      0.34     59424
weighted avg       0.26      0.51      0.35     59424



D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: 

In [17]:
sr01=sv('rbf',0.1)

Accuracy: 0.5237614431879375
Precision: 0.5293047862846002
Recall: 0.5237614431879375
F1: 0.4636601590821773
Confusion Matrix:
[[25805  4566]
 [23734  5319]]
Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.85      0.65     30371
           1       0.54      0.18      0.27     29053

    accuracy                           0.52     59424
   macro avg       0.53      0.52      0.46     59424
weighted avg       0.53      0.52      0.46     59424



In [18]:
sr1=sv('rbf',1)

Accuracy: 0.5361974959612278
Precision: 0.5354681517067794
Recall: 0.5361974959612278
F1: 0.533868491321027
Confusion Matrix:
[[18385 11986]
 [15575 13478]]
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.61      0.57     30371
           1       0.53      0.46      0.49     29053

    accuracy                           0.54     59424
   macro avg       0.54      0.53      0.53     59424
weighted avg       0.54      0.54      0.53     59424



In [19]:
sr10=sv('rbf',10)

Accuracy: 0.5204294561120086
Precision: 0.5203198980822449
Recall: 0.5204294561120086
F1: 0.5203603370521186
Confusion Matrix:
[[16278 14093]
 [14405 14648]]
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.54      0.53     30371
           1       0.51      0.50      0.51     29053

    accuracy                           0.52     59424
   macro avg       0.52      0.52      0.52     59424
weighted avg       0.52      0.52      0.52     59424



In [20]:
sr100=sv('rbf',100)

Accuracy: 0.5180903338718363
Precision: 0.5179622974032118
Recall: 0.5180903338718363
F1: 0.518006991878429
Confusion Matrix:
[[16234 14137]
 [14500 14553]]
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.53      0.53     30371
           1       0.51      0.50      0.50     29053

    accuracy                           0.52     59424
   macro avg       0.52      0.52      0.52     59424
weighted avg       0.52      0.52      0.52     59424



In [21]:
sp001=sv('poly',0.01)

Accuracy: 0.5126379913839526
Precision: 0.5296387244531768
Recall: 0.5126379913839526
F1: 0.36157931326247145
Confusion Matrix:
[[29938   433]
 [28528   525]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.99      0.67     30371
           1       0.55      0.02      0.03     29053

    accuracy                           0.51     59424
   macro avg       0.53      0.50      0.35     59424
weighted avg       0.53      0.51      0.36     59424



In [22]:
sp01=sv('poly',0.1)

Accuracy: 0.5343127355950458
Precision: 0.533892018666212
Recall: 0.5343127355950458
F1: 0.5250841467637604
Confusion Matrix:
[[20353 10018]
 [17655 11398]]
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.67      0.60     30371
           1       0.53      0.39      0.45     29053

    accuracy                           0.53     59424
   macro avg       0.53      0.53      0.52     59424
weighted avg       0.53      0.53      0.53     59424



In [23]:
sp1=sv('poly',1)

Accuracy: 0.5274636510500808
Precision: 0.527185580003824
Recall: 0.5274636510500808
F1: 0.5272167243149758
Confusion Matrix:
[[16756 13615]
 [14465 14588]]
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.55      0.54     30371
           1       0.52      0.50      0.51     29053

    accuracy                           0.53     59424
   macro avg       0.53      0.53      0.53     59424
weighted avg       0.53      0.53      0.53     59424



In [24]:
sp10=sv('poly',10)

Accuracy: 0.5181071620893914
Precision: 0.5181426248906245
Recall: 0.5181071620893914
F1: 0.5181234849287653
Confusion Matrix:
[[16004 14367]
 [14269 14784]]
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.53      0.53     30371
           1       0.51      0.51      0.51     29053

    accuracy                           0.52     59424
   macro avg       0.52      0.52      0.52     59424
weighted avg       0.52      0.52      0.52     59424



In [25]:
sp100=sv('poly',100)

Accuracy: 0.5147751750134626
Precision: 0.5147477266602655
Recall: 0.5147751750134625
F1: 0.5147606092023672
Confusion Matrix:
[[15992 14379]
 [14455 14598]]
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.53      0.53     30371
           1       0.50      0.50      0.50     29053

    accuracy                           0.51     59424
   macro avg       0.51      0.51      0.51     59424
weighted avg       0.51      0.51      0.51     59424



In [26]:
sg001=sv('sigmoid',0.01)

Accuracy: 0.5110897953688746
Precision: 0.26121277893019806
Recall: 0.5110897953688746
F1: 0.34572767247949415
Confusion Matrix:
[[30371     0]
 [29053     0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      1.00      0.68     30371
           1       0.00      0.00      0.00     29053

    accuracy                           0.51     59424
   macro avg       0.26      0.50      0.34     59424
weighted avg       0.26      0.51      0.35     59424



D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1344: 

In [27]:
sg01=sv('sigmoid',0.1)

Accuracy: 0.5167945611200861
Precision: 0.5151734834608573
Recall: 0.5167945611200861
F1: 0.5099154324759652
Confusion Matrix:
[[19215 11156]
 [17558 11495]]
Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.63      0.57     30371
           1       0.51      0.40      0.44     29053

    accuracy                           0.52     59424
   macro avg       0.52      0.51      0.51     59424
weighted avg       0.52      0.52      0.51     59424



In [28]:
sg1=sv('sigmoid',1)

Accuracy: 0.49634827679052235
Precision: 0.49610741171520467
Recall: 0.49634827679052235
F1: 0.4961712039794959
Confusion Matrix:
[[15724 14647]
 [15282 13771]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.52      0.51     30371
           1       0.48      0.47      0.48     29053

    accuracy                           0.50     59424
   macro avg       0.50      0.50      0.50     59424
weighted avg       0.50      0.50      0.50     59424



In [29]:
sg10=sv('sigmoid',10)

Accuracy: 0.4992763866451265
Precision: 0.4994395824574509
Recall: 0.4992763866451265
F1: 0.4993309909801134
Confusion Matrix:
[[15275 15096]
 [14659 14394]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.50      0.51     30371
           1       0.49      0.50      0.49     29053

    accuracy                           0.50     59424
   macro avg       0.50      0.50      0.50     59424
weighted avg       0.50      0.50      0.50     59424



In [30]:
sg100=sv('sigmoid',100)

Accuracy: 0.5001346257404415
Precision: 0.5002831720670742
Recall: 0.5001346257404415
F1: 0.5001864704324055
Confusion Matrix:
[[15320 15051]
 [14653 14400]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.50      0.51     30371
           1       0.49      0.50      0.49     29053

    accuracy                           0.50     59424
   macro avg       0.50      0.50      0.50     59424
weighted avg       0.50      0.50      0.50     59424



In [31]:
svm_res=[[sl001,sl01,sl1,sl10,sl100],[sr001,sr01,sr1,sr10,sr100],[sp001,sp01,sp1,sp10,sp100],[sg001,sg01,sg1,sg10,sg100]]
pickle_out=open("svm_res","wb")
pickle.dump(svm_res,pickle_out)
pickle_out.close()

Using SIFT features as input to SVM directly, cannot get a good result.
Most of kernels and C values are not better than random guess.